In [1]:
import pickle
import numpy as np
import torch
from scipy.stats import spearmanr
import os

In [3]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [4]:
dataset = 'cifar10'
proj_dim = 4096
method = 'trak'

In [4]:
train_index_path = f'./data/{dataset}/idx-train.pkl'

In [5]:
# Load train index
with open(train_index_path, 'rb')  as handle:
    idx_train = pickle.load(handle)
print("len(idx_train):", len(idx_train))

len(idx_train): 10000


In [6]:
# load lds subset index
mask_array_list = []
for i in range(256):
    with open(f'./data/{dataset}/lds-val/sub-idx-{i}.pkl', 'rb')  as handle:
        sub_idx_train = pickle.load(handle)
    mask_array = np.in1d(idx_train, sub_idx_train)
    mask_array_list.append(mask_array)
lds_mask_array = np.stack(mask_array_list)
print("lds_mask_array.shape:", lds_mask_array.shape)

lds_mask_array.shape: (256, 10000)


In [7]:
# load lds subset model output
loss_array_list = []
for i in range(256):
    
    for seed in [0,1,2]:
        with open(f'./saved/models/{dataset}/lds-val/index-{i}-seed-{seed}/test_CE.pkl', 'rb')  as handle:
            # -log(p/(1-p))
            loss_list = pickle.load(handle)
        margins = np.concatenate(loss_list, axis=-1)

        if (seed == 0):
            loss_array = margins
        else:
            loss_array += margins

        loss_array = loss_array/3

    loss_array_list.append(loss_array) 

lds_loss_array = np.stack(loss_array_list)
print("lds_loss_array.shape:", lds_loss_array.shape)

lds_loss_array.shape: (256, 2000)


In [8]:
print(lds_loss_array[0])


[0.00534243 0.0128518  0.743119   ... 0.00122539 0.14862019 0.01446892]


In [5]:
# load grad
train_grad_list = []
for seed in [0,1,2]:
    train_grad_seed = np.memmap(
        f'./saved/grad/{dataset}/seed-{seed}/train-{proj_dim}.npy', 
        dtype=np.float32, 
        mode='r',
        shape=(50000, proj_dim)
    )
    train_grad_list.append(train_grad_seed)
train_grad = np.stack(train_grad_list)
print("train_grad.shape:", train_grad.shape)
train_grad = torch.from_numpy(train_grad).cuda()

train_grad.shape: (3, 50000, 4096)


In [5]:
print(train_grad[0])

tensor([[   2.6007,   -4.9872,   -1.1759,  ...,    1.9265,    3.2511,
            0.9981],
        [  10.2037,   -3.1205,   38.7742,  ...,   -0.7167,   36.7600,
           25.9978],
        [   0.3561,   -0.4147,    0.8392,  ...,   -1.5259,    0.9411,
            1.0625],
        ...,
        [ -33.7301,  -67.1206,   -2.4379,  ...,  -15.1601,   35.2279,
           19.3252],
        [ -69.9211,  122.3050, -187.9740,  ...,   70.7942,   -9.9055,
          -16.9099],
        [   0.3797,   -5.9832,   -2.9847,  ...,   12.1952,   -8.8484,
            6.3969]], device='cuda:0')


In [6]:
test_grad_list = []
for seed in [0,1,2]:
    test_grad_seed = np.memmap(
        f'./saved/grad/{dataset}/seed-{seed}/test-{proj_dim}.npy', 
        dtype=np.float32, 
        mode='r',
        shape=(10000, proj_dim)
    )
    test_grad_list.append(test_grad_seed)
test_grad = np.stack(test_grad_list)
print("test_grad.shape:", test_grad.shape)
test_grad = torch.from_numpy(test_grad).cuda()

test_grad.shape: (3, 10000, 4096)


In [12]:
# load training set error
train_error_list = []

    
for seed in [0,1,2]:
    with open(f'./saved/models/{dataset}/origin/seed-{seed}/train_error.pkl', 'rb')  as handle:
        # 1-p
        error_list = pickle.load(handle)
    error_array = np.concatenate(error_list, axis=-1)

    if (seed == 0):
        train_error = error_array
    else:
        train_error += error_array

train_error = train_error/3
print("train_error.shape:", train_error.shape)

train_error_diag = np.diag(train_error)
print("train_error_diag.shape:", train_error_diag.shape)


train_error.shape: (10000,)
train_error_diag.shape: (10000, 10000)


In [7]:
# calculate the score
lds_list = []
lamb_list = [
        1e-2, 2e-2, 5e-2,
        1e-1, 2e-1, 5e-1,
        1e0, 2e0, 5e0,
        1e1, 2e1, 5e1,
        1e2, 2e2, 5e2,
        1e3, 2e3, 5e3, 
        1e4, 2e4, 5e4, 
        1e5, 2e5, 5e5, 
        1e6, 2e6, 5e6, 
    ]

rs_list = []
ps_list = []
best_scores = None
best_lds = -np.inf

In [8]:
from pathlib import Path
# corr = eval_correlations(scores, '.')
tmp_path = './tmp'
masks_path = Path(tmp_path).joinpath('mask.npy')
masks = torch.as_tensor(np.load(masks_path, mmap_mode='r')).float()
print(masks.shape)

margins_path = Path(tmp_path).joinpath('val_margins.npy')
margins = torch.as_tensor(np.load(margins_path, mmap_mode='r'))
print(margins.shape)

torch.Size([6011, 50000])
torch.Size([6011, 10000])


/tmp/ipykernel_2710358/233794002.py:5: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at /opt/conda/conda-bld/pytorch_1699449229234/work/torch/csrc/utils/tensor_numpy.cpp:206.)
  masks = torch.as_tensor(np.load(masks_path, mmap_mode='r')).float()


In [12]:
from tqdm import tqdm
for lamb in lamb_list:

    scores_list = []

    for seed in [0,1,2]:

        train_grad_seed = train_grad[seed]
        test_grad_seed = test_grad[seed]
        kernel = train_grad_seed.T@train_grad_seed
        
        kernel_ = kernel + lamb*torch.eye(kernel.shape[0]).cuda()
        kernel_ = torch.linalg.inv(kernel_)  

        if method == 'trak':
            scores_seed = test_grad_seed@((train_grad_seed@kernel_).T)
            
        scores_seed = scores_seed.cpu().numpy()
        scores_list.append(scores_seed)
    
    scores = np.stack(scores_list)
    scores = scores.mean(axis=0)
    # scores = scores@train_error_diag

    val_inds = np.arange(10000)
    preds = masks @ scores.T
    rs = []
    ps = []
    for ind, j in tqdm(enumerate(val_inds)):
        r, p = spearmanr(preds[:, ind], margins[:, j])
        rs.append(r)
        ps.append(p)
    rs, ps = np.array(rs), np.array(ps)
    print(f'Correlation: {rs.mean():.3f} (avg p value {ps.mean():.6f})')

10000it [00:12, 791.43it/s]


Correlation: -0.000 (avg p value 0.487322)


10000it [00:12, 791.53it/s]


Correlation: -0.000 (avg p value 0.487322)


10000it [00:12, 781.64it/s]


...

Correlation: 0.000 (avg p value 0.487569)


10000it [00:12, 781.56it/s]


Correlation: 0.000 (avg p value 0.487662)


10000it [00:12, 784.03it/s]

Correlation: 0.000 (avg p value 0.487761)


In [15]:
for lamb in lamb_list:

    scores_list = []

    for seed in [0,1,2]:

        train_grad_seed = train_grad[seed]
        test_grad_seed = test_grad[seed]
        kernel = train_grad_seed.T@train_grad_seed
        
        kernel_ = kernel + lamb*torch.eye(kernel.shape[0]).cuda()
        kernel_ = torch.linalg.inv(kernel_)  

        if method == 'trak':
            scores_seed = test_grad_seed@((train_grad_seed@kernel_).T)
            
        scores_seed = scores_seed.cpu().numpy()
        scores_list.append(scores_seed)
    
    scores = np.stack(scores_list)
    scores = scores.mean(axis=0)
    scores = scores@train_error_diag

    margins = lds_loss_array
    infl_est_ = -scores
    preds = lds_mask_array @ infl_est_.T

    # compute lds
    rs = []
    ps = []
    for ind in range(2000):
        r, p = spearmanr(preds[:, ind], margins[:, ind])
        rs.append(r)
        ps.append(p)
    rs, ps = np.array(rs), np.array(ps)
    print(f'Correlation: {rs.mean():.3f} (avg p value {ps.mean():.6f})')
    
    rs_list.append(rs.mean())   
    ps_list.append(ps.mean())

    if rs.mean()>best_lds:
        best_scores = scores
        best_lds = rs.mean()

print(f'best_lds: {best_lds:.3f}')



Correlation: -0.001 (avg p value 0.496782)
Correlation: -0.001 (avg p value 0.496782)
Correlation: -0.001 (avg p value 0.496782)
Correlation: -0.001 (avg p value 0.496639)
Correlation: -0.001 (avg p value 0.497024)
Correlation: -0.001 (avg p value 0.496754)
Correlation: -0.001 (avg p value 0.496501)
Correlation: -0.001 (avg p value 0.496666)
Correlation: -0.001 (avg p value 0.496913)
Correlation: -0.001 (avg p value 0.496929)
Correlation: -0.001 (avg p value 0.496910)
Correlation: -0.001 (avg p value 0.496853)
Correlation: -0.001 (avg p value 0.497186)
Correlation: -0.001 (avg p value 0.497028)
Correlation: -0.001 (avg p value 0.496702)
Correlation: -0.001 (avg p value 0.496955)
Correlation: -0.001 (avg p value 0.497747)
Correlation: -0.001 (avg p value 0.499855)


KeyboardInterrupt: 